In [4]:
!pip install scikit-learn
!pip install scikit-uplift
!pip install xgboost

     |████████████████████████████████| 6.8 MB 14.0 MB/s eta 0:00:01
ERROR: azureml-train-automl-runtime 1.5.0 has requirement pandas<=0.23.4,>=0.21.0, but you'll have pandas 1.0.5 which is incompatible.
ERROR: azureml-train-automl-runtime 1.5.0 has requirement scikit-learn<=0.20.3,>=0.19.0, but you'll have scikit-learn 0.23.1 which is incompatible.
ERROR: azureml-automl-runtime 1.5.0.post1 has requirement pandas<=0.23.4,>=0.21.0, but you'll have pandas 1.0.5 which is incompatible.
ERROR: azureml-automl-runtime 1.5.0.post1 has requirement scikit-learn<=0.20.3,>=0.19.0, but you'll have scikit-learn 0.23.1 which is incompatible.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [5]:
import catboost as cb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from pandas import read_csv
import gc
import pandas as pd
import numpy as np
from datetime import datetime

from xgboost import XGBClassifier
from sklift.models import ClassTransformation

from sklift.metrics import uplift_at_k
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
data = read_csv('dataset.csv')

In [7]:
y = data['response_att']
X = data.drop(columns=['response_att'])


In [8]:
xgb_est_params = {
    'max_depth':2,
    'learning_rate': 0.2, 
    'n_estimators': 100,
    'nthread':40,
    'n_gpus':0,
    'seed':42
}

estimator = XGBClassifier(
    **xgb_est_params
)

uplift_model_cl_tr = ClassTransformation(
    estimator=estimator
)

In [14]:
X.drop(columns=['group'])

,Unnamed: 0,CardHolder,age,cheque_count_12m_g20,cheque_count_12m_g21,cheque_count_12m_g25,cheque_count_12m_g32,cheque_count_12m_g33,cheque_count_12m_g38,cheque_count_12m_g39,...,sale_sum_6m_g24,sale_sum_6m_g25,sale_sum_6m_g26,sale_sum_6m_g32,sale_sum_6m_g33,sale_sum_6m_g44,sale_sum_6m_g54,stdev_days_between_visits_15d,stdev_discount_depth_15d,stdev_discount_depth_1m
0,0,16095858,47.0,3.0,22.0,19.0,3.0,28.0,8.0,7.0,...,3141.25,356.67,237.25,283.84,3648.23,1195.37,535.42,1.7078,0.2798,0.3008
1,1,15906620,57.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,...,113.39,62.69,58.71,87.01,179.83,0.00,122.98,0.0000,0.0000,0.0000
2,2,16495466,38.0,7.0,0.0,15.0,4.0,9.0,5.0,9.0,...,1239.19,533.46,83.37,593.13,1217.43,1336.83,3709.82,0.0000,0.0000,0.0803
3,3,16570217,65.0,6.0,3.0,25.0,2.0,10.0,14.0,11.0,...,139.68,1849.91,360.40,175.73,496.73,172.58,1246.21,0.0000,0.0000,0.0000
4,4,16346871,61.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,...,226.98,168.05,461.37,0.00,237.93,225.51,995.27,1.4142,0.3495,0.3495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687024,687024,15705403,35.0,0.0,0.0,4.0,0.0,2.0,0.0,1.0,...,550.09,669.33,111.87,0.00,330.96,1173.84,119.99,2.6458,0.3646,0.3282
687025,687025,16586008,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,28.01,0.0000,0.0000,0.0000
687026,687026,15693616,36.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,449.01,0.00,0.0000,0.0000,0.0000
687027,687027,15915514,37.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.00,46.72,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.0000


In [15]:
uplift_model_cl_tr.fit(
    X=X.drop(columns=['group']),
    y=y,
    treatment=X['group']
)

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



ClassTransformation(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.2,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_gpus=0, n_jobs=1, nthread=40,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=None, subsample=1,
       verbosity=1))

In [16]:
df_feat_imp = pd.DataFrame([
    uplift_model_cl_tr\
        .estimator\
        .get_booster()\
        .get_score(importance_type='gain')
        ]
).T.reset_index()

df_feat_imp.columns =\
    ["feature", "value"]
    
df_feat_imp\
    .sort_values('value', ascending=False)

,feature,value
3,perdelta_days_between_visits_15_30d,1073.210511
0,k_var_days_between_visits_15d,986.238432
4,k_var_days_between_visits_1m,855.204125
9,k_var_days_between_visits_3m,568.014705
5,cheque_count_6m_g41,512.041015
...,...,...
86,k_var_sku_price_3m_g54,10.480846
89,k_var_count_per_cheque_1m_g27,10.282679
79,k_var_disc_share_12m_g32,10.028054
90,sale_count_6m_g32,9.722181


In [20]:
list(df_feat_imp\
    .sort_values('value', ascending=False)['feature'])[:25]

['perdelta_days_between_visits_15_30d',
 'k_var_days_between_visits_15d',
 'k_var_days_between_visits_1m',
 'k_var_days_between_visits_3m',
 'cheque_count_6m_g41',
 'cheque_count_6m_g40',
 'response_sms',
 'cheque_count_6m_g48',
 'cheque_count_12m_g38',
 'cheque_count_6m_g32',
 'response_viber',
 'k_var_count_per_cheque_6m_g27',
 'cheque_count_6m_g25',
 'cheque_count_12m_g32',
 'k_var_disc_share_15d_g24',
 'k_var_disc_share_6m_g24',
 'food_share_1m',
 'k_var_cheque_3m',
 'cheque_count_6m_g38',
 'cheque_count_3m_g25',
 'age',
 'food_share_15d',
 'cheque_count_12m_g41',
 'stdev_days_between_visits_15d',
 'k_var_sku_price_6m_g26']